In [ ]:
import pylatex
import mysql.connector
import pandas
import os
import numpy as np
import matplotlib.pyplot as plt
import subprocess

con = mysql.connector.connect(
    host = "giniewicz.it",
    user = "team19",
    password = "te@mzaiq",
    database = "team19"
)

cursor = con.cursor()
# Zapytanie SQL do obliczenia średnich ocen dla realizowanych wycieczek
query = '''
SELECT Rodzaje_wycieczek.Nazwa AS Wycieczka, 
       AVG(Oceny.Ocena) AS Srednia_Ocena
FROM Oceny
JOIN Rezerwacje ON Oceny.id_rezerwacje = Rezerwacje.id_rezerwacje
JOIN Realizowane_wycieczki ON Rezerwacje.id_wycieczki = Realizowane_wycieczki.id_wycieczki
JOIN Rodzaje_wycieczek ON Realizowane_wycieczki.id_rodzaj = Rodzaje_wycieczek.id_rodzaj
GROUP BY Rodzaje_wycieczek.Nazwa
ORDER BY Srednia_Ocena DESC;
'''

# Wykonanie zapytania
cursor.execute(query)
results = cursor.fetchall()

# Zamknięcie połączenia
con.close()

# Przygotowanie danych do wykresu
wycieczki = [row[0] for row in results]
srednie_oceny = [row[1] for row in results]

# Generowanie pastelowych kolorów dla słupków
kolory = plt.cm.Set2(np.linspace(0, 1, len(wycieczki)))
kolory[0] = np.array([1, 0.6, 0.6, 1])  
kolory[-1] = np.array([0.7, 0.4, 1, 1])  

# Tworzenie wykresu
plt.figure(figsize=(10, 6))
plt.barh(wycieczki, srednie_oceny, color=kolory)
plt.xlabel('Średnia Ocena', fontweight='bold', fontsize=12)
plt.ylabel('Rodzaj wycieczki', fontweight='bold', fontsize=12)
plt.title('Średnie Oceny Wycieczek', fontsize=16, fontweight='bold', loc='center')
plt.gca().invert_yaxis()  
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()

plot_filename = "Oceny.png"
plt.savefig(plot_filename)
plt.close()
